# A handmade agent for math

In this notebook, we will go through steps required to define an Ebiose agent by hand and run it with the LangGraph backend. 

We will implement the following simple agent which aims at solving math problems:

<div align="center">


```mermaid
graph LR
     Startnode[StartNode] --> Solver(solver)
     Solver(solver) --> Verifier(verifier)
     Verifier(verifier) -->|correct| Endnode[EndNode]
     Verifier(verifier) -->|incorrect| Solver(solver)
```
**Example of a graph made to solve math problems**
</div>

In [1]:
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()

True

## Defining agent IO

The input and output of an Ebiose agent are user-defined with Pydantic models, such as:

In [2]:
from pydantic import BaseModel, Field, ConfigDict

class AgentInput(BaseModel):
      math_problem: str = Field(..., description="The mathematical word problem to solve")

class AgentOutput(BaseModel):
      """The expected final output to the mathematical problem."""
      rationale: str = Field(..., description="The rationale for the solution")
      solution: int = Field(..., description="The solution to the problem.")

## Building the agent

An Ebiose agent is mainly defined by a `Graph` instance, which defines the workflow of the agent. Each graph is composed of nodes and edges.


### The shared context prompt

Every graph has a so-called "shared context prompt" which will be given as context to each LLMNode composing the graph. It allows to share high-level context to each LLM node. 

Prompts can feature placeholders in curly braces `{placeholder}` which will be automatically replaced with input values when running the agent. The only authorized placeholders are the fields of the `AgentInput`'s, *i.e.$, here the field `math_problem`. 

In [3]:
shared_context_prompt = """
Your are part of a multi-node agent that solves math problems.
The agent has two main nodes: the solver node and the verifier node.
The solver node solves the math problem and the verifier node verifies the solution
given by the solver node. If it is incorrect, the verifier node provides insights
back to the solver node so that it improves the solution.
"""

### The solver node

Here, we define a first LLMNode which will be in charge of giving a very first solution to the math problem at stake. 

To do so, we write its prompt and create a `LLMNode` instance, by providing the prompt and an id, a name and a purpose.

In [4]:
from ebiose.core.engines.graph_engine.nodes import LLMNode

solver_prompt = """
Your are the Solver node. You must solve the given math problem.
"""


solver_node = LLMNode(
    id="solver",
    name="Solver",
    purpose="solve the math problem",
    prompt=solver_prompt,
)

### The verifier node

We here defined a second `LLMNode` which will be in charge of verifying the solution provided by the Solver node. 

In [5]:
verifier_prompt = """
You are the Verified node.
Based on the solution provided by the Solver node,
you must decide whether the solution is correct or not.
If the solution is incorrect, explain why and provide insights back
to the solver node so that it improves the solution.
"""

verifier_node = LLMNode(
    id="verifier",
    name="Verifier",
    purpose="verify the math problem",
    prompt=verifier_prompt,
)

### The start and end nodes

The `StartNode` and `EndNode` are necessary to represent the entry point and output point of the graph, and have no other extra role to play.

In [6]:
from ebiose.core.engines.graph_engine.nodes import StartNode, EndNode

start_node = StartNode()
end_node = EndNode()

### The graph

Having defined the nodes, we can now create the graph, add the nodes and connect them with edges.

In [7]:
from ebiose.core.engines.graph_engine.graph import Graph
from ebiose.core.engines.graph_engine.edge import Edge

math_graph = Graph(shared_context_prompt=shared_context_prompt)

# adding nodes
math_graph.add_node(start_node)
math_graph.add_node(solver_node)
math_graph.add_node(verifier_node)
math_graph.add_node(end_node)

# adding edges
# from start to solver
math_graph.add_edge(Edge(start_node_id=start_node.id, end_node_id=solver_node.id))
# from solver to verifier
math_graph.add_edge(Edge(start_node_id=solver_node.id, end_node_id=verifier_node.id))
# from verifier to end,  if the condition is correct
math_graph.add_edge(Edge(start_node_id=verifier_node.id, end_node_id=end_node.id, condition="correct"))
# from verifier to solver, if the condition is incorrect
math_graph.add_edge(Edge(start_node_id=verifier_node.id, end_node_id=solver_node.id, condition="incorrect"))

Note that some of the added edges have a `condition` attribute : 
```python 
Edge(start_node_id=verifier_node.id, end_node_id=end_node.id, condition="correct")
Edge(start_node_id=verifier_node.id, end_node_id=solver_node.id, condition="incorrect")
```
which means that, the Verifier node will have to decide if the solution given by the Solver node is correct or incorrect. If it is judged as correct, then the next node to be executed will be the End node. Else, the next will node will be the Solver node again, so that it tries to correct its initial solution based on the feedback provided by the Verifier node. 

Finally, we can print the created graph as a Mermaid string for visualisation, on [Mermaid.live](https://mermaid.live/) for instance.

In [8]:
print(math_graph.to_mermaid_str())

graph TD
	Start_Node[start_node] --> Solver(Solver)
	Solver(Solver) --> Verifier(Verifier)
	Verifier(Verifier) -->|correct| End_Node[end_node]
	Verifier(Verifier) -->|incorrect| Solver(Solver)



Finally, we can create an Ebiose agent from the created graph.

In [ ]:
from ebiose.core.agent import Agent
from ebiose.core.agent_engine_factory import AgentEngineFactory


math_graph_engine = AgentEngineFactory.create_engine(
    engine_type="langgraph_engine",
    configuration={"graph": math_graph.model_dump()},
    input_model=AgentInput,
    output_model=AgentOutput,
    model_endpoint_id="gpt-4o-mini"
)

math_agent = Agent(
    name="Math Agent",
    description="An agent that solves math problems",
    agent_engine=math_graph_engine,
)

pprint(math_agent.model_dump())

{'agent_engine': {'configuration': {'graph': {'description': '',
                                              'edges': [{'condition': None,
                                                         'end_node_id': 'solver',
                                                         'start_node_id': 'start_node'},
                                                        {'condition': None,
                                                         'end_node_id': 'verifier',
                                                         'start_node_id': 'solver'},
                                                        {'condition': 'correct',
                                                         'end_node_id': 'end_node',
                                                         'start_node_id': 'verifier'},
                                                        {'condition': 'incorrect',
                                                         'end_node_id': 'solver',
                          

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


## Running the agent
To run the agent, we now just have to create an instance of `AgentInput` with a math problem to be solved.

In [10]:
agent_input = math_agent.agent_engine.input_model(
    math_problem="Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"
)
# we could have also used AgentInput(math_problem=...) as well

Finally, we pass it to the `run` method of the agent which will return the last_response

In [ ]:
from ebiose.compute_intensive_batch_processor.compute_intensive_batch_processor import ComputeIntensiveBatchProcessor
from ebiose.core.model_endpoint import ModelEndpoint

ComputeIntensiveBatchProcessor.initialize()
BUDGET = 0.01

import asyncio
import nest_asyncio
nest_asyncio.apply()
response = asyncio.run(math_agent.run(agent_input))
pprint(response)

2025-03-19 00:33:10.436 | DEBUG    | ebiose.core.agent:run:63 - Error while running agent agent-369eae52-1a60-4f33-9ca1-42650e7d1e02: name 'BaseModel' is not defined


None
